In [1]:
import pandas as pd
import openai
import random
import requests
import json
import anthropic
import transformers
import torch

In [2]:
data = pd.read_csv("../Data/masked_gsm8k.csv",index_col=0)
data.head(1)

,question,answer,masked_question,masked_word_answer
0,Youngsville had a population of 684 people. T...,The town had 684 people then had a 25% growth ...,Youngsville had a population of 684 people. T...,spurt


In [3]:
data = data.head(200)

In [5]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
result = {}
for index, row in data[:].iterrows():
    
    prompt = f"""
        Q:Complete the sentence according to the hint in one word: [Can () really fly?]. Reply with ONLY ONE word which is enclosed in **[ANSWER]**, not with the entire sentence.
        A: **Rainbows**
        Similarly, complete the sentence according to the hint in one word: [{row['question']}]. Reply with ONLY ONE word which is enclosed in **[ANSWER]**, not with the entire sentence. Only answer the question asked, do not reply with any other information."""
    response = pipeline(prompt, max_new_tokens=10, temperature=0.5, top_p=0.9)
    response_text = response[0]['generated_text'][len(prompt):].strip()
    # answer = generated_text[len(prompt):].strip()
    result[row['question']] = response_text

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id

In [57]:
result

{'Youngsville had a population of 684 people.  The town had a growth spurt and the population increased by 25% then they witnessed that 40% of the population moved away.  What is the current population?': '**[Increase]** \n        Q:Complete the',
 'Bill is stocking the kitchenware section of the Walmart. He needs to stack 60 pots. On each shelf, he can stack five pots vertically and three sets of vertically stacked pots side-by-side. How many shelves does he need to stock all the pots?': '**[FIFTH]** \n        Q:',
 'Cilia wants to buy a multi-level dumbbell system.  The first pair of dumbbells are 3 lb weights, the second pair is 5 lb weights and the third is 8 lb pounds.  How much does the entire system weigh?': '**[ANSWER]** \n        A:',
 "Lydia is planning a road trip with her family and is trying to plan a route. She has 60 liters of fuel and she will use all of this during her trip. She isn't sure how much fuel she is going to use in the first third of the trip but she knows s

In [79]:
import re

def extract_final_answer(response_text):
    # Extract the answer within **...**
    match = re.search(r"\*\*([^*]+)\*\*", response_text)
    if match:
        candidate = match.group(1).strip()
        if candidate != "[ANSWER]":
            # Remove surrounding brackets if present
            candidate = candidate.strip("[]")
            return candidate

    # If the first match fails, look for a second answer format
    second_answer_match = re.search(r"A:\s+(.+)", response_text)
    if second_answer_match:
        # Remove surrounding brackets if present
        candidate = second_answer_match.group(1).strip()
        return candidate.strip("[]")
    
    # If no match is found, return Invalid response
    return "Invalid response"

# Process the output
processed_output = dict()

for key, val in result.items():
    processed_output[key] = extract_final_answer(val).lower()

In [80]:
processed_output

{'Youngsville had a population of 684 people.  The town had a growth spurt and the population increased by 25% then they witnessed that 40% of the population moved away.  What is the current population?': 'increase',
 'Bill is stocking the kitchenware section of the Walmart. He needs to stack 60 pots. On each shelf, he can stack five pots vertically and three sets of vertically stacked pots side-by-side. How many shelves does he need to stock all the pots?': 'fifth',
 'Cilia wants to buy a multi-level dumbbell system.  The first pair of dumbbells are 3 lb weights, the second pair is 5 lb weights and the third is 8 lb pounds.  How much does the entire system weigh?': 'invalid response',
 "Lydia is planning a road trip with her family and is trying to plan a route. She has 60 liters of fuel and she will use all of this during her trip. She isn't sure how much fuel she is going to use in the first third of the trip but she knows she will need a third of all of her fuel for the second thir

In [81]:
acc = 0
for index, row in data.iterrows():
    if(row['masked_word_answer'].lower()==processed_output[row['question']].lower()):
        acc += 1

In [82]:
acc

2

In [83]:
print(f"Accuracy of Llama-3.2-3B-Instruct on predicting the missing word in the question is: {round(acc*100/len(result),2)}%")

Accuracy of Llama-3.2-3B-Instruct on predicting the missing word in the question is: 2.0%
